In [1]:
import numpy as np
import pandas as pd

from asteroid_dataframe import load_ast_data, calc_ast_data, spline_ast_vec_df, spline_ast_dir
from asteroid_data import orbital_element_batch
from ztf_data import load_ztf_det_all, ztf_reindex, load_ztf_nearest_ast, make_ztf_near_elt, calc_hit_freq
from astro_utils import deg2dist

from typing import Dict

In [2]:
n0=1
n1=65

mjd0 = 58270.0
mjd1 = 58906.0

progbar=True

In [58]:
# Load data in this date range
df_ast_load, df_earth_load, df_sun_load = load_ast_data(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1, progbar=progbar)

Loading asteroid data from n0=1 to n1=65 in 1 blocks...


In [59]:
# df_ast_load

In [60]:
# Orbital elements for first 64 asteroids (dict of numpy arrays)
elts = orbital_element_batch(np.arange(1, 65))

In [5]:
# Calculate vectors from these elements on the fly with calc_ast_data
df_ast_calc, df_earth_calc, df_sun_calc = calc_ast_data(elts=elts, mjd0=mjd0, mjd1=mjd1, element_id=None)

In [6]:
# df_ast_calc

## Splined Direction at ZTF Dates

In [63]:
ztf, mjd_unq = load_ztf_det_all()

In [64]:
ztf

,ObjectID,CandidateID,mjd,ra,dec,mag_psf,mag_app,mag_nr,asteroid_prob
0,b'ZTF18acebhfp',676397301515010013,58430.397303,41.357345,58.879488,18.339935,18.946699,15.781000,0.865682
1,b'ZTF18abodmwk',596403415715010014,58350.403414,30.969721,65.305308,18.996313,19.010401,16.315001,0.855504
2,b'ZTF18abodmwk',626428345715010011,58380.428345,30.969705,65.305294,18.791924,18.935900,16.315001,0.855504
3,b'ZTF18abodmwk',630507595715015045,58384.507593,30.969940,65.305305,19.037745,19.260401,16.315001,0.855504
4,b'ZTF18abodmwk',618384965715010022,58372.384965,30.969643,65.305179,19.139240,19.220200,16.315001,0.855504
...,...,...,...,...,...,...,...,...,...
5697957,b'ZTF20aareruw',1151532523515015015,58905.532523,253.007910,55.485537,19.354780,19.192400,21.096001,0.608023
5697958,b'ZTF20aarerwx',1151533002615015009,58905.533009,232.886408,53.509617,19.636295,19.687099,22.781000,0.559474
5697959,b'ZTF20aarerww',1151533002115010003,58905.533009,236.167899,54.618457,19.685007,19.957001,22.848000,0.392662
5697960,b'ZTF20aarervr',1151526063515015015,58905.526065,286.235286,33.876902,18.849386,19.049299,20.461000,0.517241


In [67]:
# mjd for the spline output
mjd = mjd_unq

In [68]:
df_ast_load_out, df_earth_load_out, df_sun_load_out = spline_ast_vec_df(df_ast=df_ast_load, df_earth=df_earth_load, df_sun=df_sun_load, mjd=mjd, include_elts=False)

In [69]:
# df_ast_load_out

In [70]:
df_ast_calc_out, df_earth_calc_out, df_sun_calc_out = spline_ast_vec_df(df_ast=df_ast_calc, df_earth=df_earth_calc, df_sun=df_sun_calc, mjd=mjd, include_elts=False)

In [71]:
# df_ast_calc_out

## Assemble Batch of Asteroids Near Orbital Elements

In [17]:
ztf_ast = load_ztf_nearest_ast()

In [18]:
# Asteroid numbers and hit counts
ast_num, hit_count = calc_hit_freq(ztf=ztf_ast, thresh_sec=2.0)

# Sort the hit counts in descending order and find the top 64
idx = np.argsort(hit_count)[::-1][0:64]

# Extract the asteroid number and hit count for this batch
ast_nums_best = ast_num[idx]
hit_count_best = hit_count[idx]

In [19]:
# Orbital elements for best asteroids (dict of numpy arrays)
elts_best = orbital_element_batch(ast_nums_best)

# Extract the epoch
epoch = elts['epoch'][0]

In [20]:
# Compute mjd0 and mjd1 from mjd_unq
mjd0 = np.floor(np.min(mjd_unq))
mjd1 = np.ceil(np.max(mjd_unq))
# other arguments
progbar=True
element_id = ast_nums_best

# Calculate positions in this date range, sapmled daily
df_ast_daily, df_earth_daily, df_sun_daily = calc_ast_data(elts=elts, mjd0=mjd0, mjd1=mjd1, element_id=element_id)

In [21]:
df_ast_daily

,element_id,mjd,qx,qy,qz,vx,vy,vz
0,51921,58270.0,-2.389132,0.797218,0.465399,-0.003559,-0.010556,0.000323
1,51921,58271.0,-2.392670,0.786654,0.465718,-0.003517,-0.010570,0.000314
2,51921,58272.0,-2.396166,0.776077,0.466028,-0.003475,-0.010584,0.000306
3,51921,58273.0,-2.399619,0.765486,0.466330,-0.003432,-0.010597,0.000298
4,51921,58274.0,-2.403030,0.754882,0.466624,-0.003390,-0.010611,0.000290
...,...,...,...,...,...,...,...,...
40699,72911,58901.0,-1.088110,-2.586812,-0.056603,0.008715,-0.004905,0.000084
40700,72911,58902.0,-1.079388,-2.591701,-0.056519,0.008729,-0.004871,0.000085
40701,72911,58903.0,-1.070652,-2.596554,-0.056433,0.008743,-0.004836,0.000086
40702,72911,58904.0,-1.061901,-2.601374,-0.056347,0.008758,-0.004802,0.000087


In [22]:
# Spline positions at ztf times
mjd = mjd_unq
df_ast, df_earth, df_sun = spline_ast_vec_df(df_ast=df_ast_daily, df_earth=df_earth_daily, df_sun=df_sun_daily, mjd=mjd, include_elts=False)

In [23]:
df_ast

,element_id,mjd,qx,qy,qz,vx,vy,vz
0,733,58270.170417,-2.389738,0.795418,0.465454,-0.003552,-0.010559,0.000321
1,733,58270.171343,-2.389741,0.795409,0.465454,-0.003552,-0.010559,0.000321
2,733,58270.315625,-2.390253,0.793885,0.465501,-0.003546,-0.010561,0.000320
3,733,58270.317986,-2.390262,0.793860,0.465501,-0.003546,-0.010561,0.000320
4,733,58270.318472,-2.390263,0.793855,0.465502,-0.003546,-0.010561,0.000320
...,...,...,...,...,...,...,...,...
6215099,324582,58905.531169,-1.048475,-2.608685,-0.056214,0.008779,-0.004749,0.000088
6215100,324582,58905.531620,-1.048471,-2.608688,-0.056214,0.008779,-0.004749,0.000088
6215101,324582,58905.532072,-1.048467,-2.608690,-0.056214,0.008779,-0.004749,0.000088
6215102,324582,58905.532523,-1.048463,-2.608692,-0.056214,0.008779,-0.004749,0.000088


In [24]:
df_dir = spline_ast_dir(df_ast=df_ast, df_earth=df_earth, site_name='palomar')

In [25]:
df_dir

,element_id,mjd,ra,dec,ux,uy,uz,delta
0,733,58270.170417,145.389620,24.224476,-0.750561,0.638449,0.170415,2.731457
1,733,58270.171343,145.389930,24.224339,-0.750565,0.638445,0.170414,2.731468
2,733,58270.315625,145.438630,24.202949,-0.751131,0.637803,0.170321,2.733176
3,733,58270.317986,145.439433,24.202599,-0.751140,0.637793,0.170320,2.733204
4,733,58270.318472,145.439598,24.202527,-0.751142,0.637791,0.170319,2.733210
...,...,...,...,...,...,...,...,...
6215099,324582,58905.531169,267.226879,-24.488156,-0.044029,-0.998855,-0.018729,3.004458
6215100,324582,58905.531620,267.227021,-24.488160,-0.044027,-0.998855,-0.018729,3.004452
6215101,324582,58905.532072,267.227164,-24.488164,-0.044025,-0.998855,-0.018729,3.004447
6215102,324582,58905.532523,267.227307,-24.488168,-0.044022,-0.998855,-0.018729,3.004441


In [26]:
ztf

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob
0,b'ZTF18acebhfp',676397301515010013,14490,58430.397303,41.357345,58.879488,0.387942,0.653853,0.649598,18.946699,0.865682
1,b'ZTF18abodmwk',596403415715010014,5831,58350.403414,30.969721,65.305308,0.358224,0.558644,0.748059,19.010401,0.855504
2,b'ZTF18abodmwk',626428345715010011,10614,58380.428345,30.969705,65.305294,0.358224,0.558644,0.748059,18.935900,0.855504
3,b'ZTF18abodmwk',630507595715015045,11250,58384.507593,30.969940,65.305305,0.358223,0.558645,0.748059,19.260401,0.855504
4,b'ZTF18abodmwk',618384965715010022,9040,58372.384965,30.969643,65.305179,0.358226,0.558644,0.748058,19.220200,0.855504
...,...,...,...,...,...,...,...,...,...,...,...
5459014,b'ZTF20aareruw',1151532523515015015,97109,58905.532523,253.007910,55.485537,-0.165587,-0.169403,0.971537,19.192400,0.608023
5459015,b'ZTF20aarerwx',1151533002615015009,97110,58905.533009,232.886408,53.509617,-0.358833,-0.115301,0.926253,19.687099,0.559474
5459016,b'ZTF20aarerww',1151533002115010003,97110,58905.533009,236.167899,54.618457,-0.322375,-0.116973,0.939357,19.957001,0.392662
5459017,b'ZTF20aarervr',1151526063515015015,97098,58905.526065,286.235286,33.876902,0.232120,-0.509626,0.828494,19.049299,0.517241


In [27]:
thresh_deg = 1.0
ztf_tbl = make_ztf_near_elt(ztf=ztf, df_dir=df_dir, thresh_deg=thresh_deg)

In [28]:
ztf_tbl.keys()

dict_keys([733, 1476, 1803, 2015, 2294, 3426, 3904, 4133, 5179, 5817, 6271, 6455, 9229, 9262, 9936, 10032, 10368, 10555, 11952, 13706, 14348, 14692, 15786, 17262, 18181, 19794, 25318, 26793, 27860, 30935, 31539, 31841, 43796, 48453, 49627, 51921, 53516, 59123, 59244, 59245, 62002, 72911, 73961, 76841, 77774, 85295, 85937, 90777, 92136, 103501, 104194, 112578, 114238, 120355, 122263, 134815, 142999, 161094, 178441, 203722, 253246, 306781, 313521, 324582])

In [29]:
ztf_i = ztf_tbl[733]
ztf_i

,ObjectID,TimeStampID,mjd,ra,dec,ux,uy,uz
1800,b'ZTF18ablqfuo',22567,58493.546840,234.412901,-12.778610,-0.567526,-0.815628,0.112540
784,b'ZTF19aaaqkvm',23454,58506.539051,237.731744,-14.277733,-0.517393,-0.849922,0.099683
1410,b'ZTF19aaocfcy',27722,58574.416979,253.755145,-16.126091,-0.268736,-0.956676,0.112037
153,b'ZTF19aapqdba',30558,58606.459433,251.596553,-16.624811,-0.302509,-0.947974,0.099162
90073,b'ZTF19abjauqo',40863,58692.161829,238.884711,-19.919738,-0.485844,-0.874012,0.007584
90274,b'ZTF19abjawja',40863,58692.161829,238.689957,-19.533874,-0.489759,-0.871753,0.013515
90279,b'ZTF19abjaumm',40865,58692.162731,237.870253,-19.284000,-0.501999,-0.864739,0.014957
90281,b'ZTF19abjaumm',40863,58692.161829,237.870180,-19.283972,-0.502000,-0.864738,0.014957
90333,b'ZTF19abjauqi',40863,58692.161829,239.177944,-20.066879,-0.481269,-0.876552,0.006055
93275,b'ZTF19abjbymw',40950,58692.210903,238.315861,-19.366528,-0.495516,-0.868467,0.015092


In [30]:
thresh_dist = deg2dist(thresh_deg)
mask_test = (ztf_ast.nearest_ast_num == 733) & (ztf_ast.nearest_ast_dist < thresh_dist)
ztf_test = ztf_ast[mask_test]
ztf_test

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob,nearest_ast_num,nearest_ast_dist,ast_ra,ast_dec,ast_ux,ast_uy,ast_uz
137850,b'ZTF19abjphnr',939445724715015021,41296,58693.445729,44.285479,33.579335,0.596406,0.753718,0.276059,15.9857,1.0,733,0.000004,44.285291,33.579191,0.596409,0.753717,0.276058
162589,b'ZTF19abjqzrj',939485674715015012,41349,58693.485671,44.293555,33.585205,0.596283,0.753793,0.276120,15.5081,1.0,733,0.000004,44.293377,33.585051,0.596286,0.753791,0.276118
172887,b'ZTF19abkkmjx',934476402915015003,40253,58688.476401,43.232256,32.855154,0.612042,0.743715,0.268871,16.0814,1.0,733,0.000004,43.232065,32.855006,0.612045,0.743713,0.268869
175153,b'ZTF19abkkmjx',934476861615015007,40254,58688.476863,43.232370,32.855227,0.612041,0.743716,0.268872,16.0991,1.0,733,0.000004,43.232164,32.855074,0.612044,0.743714,0.268870
115573,b'ZTF19ablpetd',942451704715015001,42140,58696.451701,44.889994,34.015204,0.587237,0.759236,0.280557,15.7284,1.0,733,0.000004,44.889810,34.015059,0.587240,0.759235,0.280555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003982,b'ZTF20aanbfry',1139159645615015005,93432,58893.159641,38.972055,36.841868,0.622190,0.700314,0.349914,15.5513,1.0,733,0.000003,38.971815,36.841881,0.622192,0.700311,0.349915
5125872,b'ZTF20aaoeplv',1142123785315015008,94493,58896.123785,39.695137,36.790572,0.616202,0.707510,0.346013,15.3835,1.0,733,0.000003,39.694888,36.790585,0.616204,0.707508,0.346014
5142573,b'ZTF20aaoggbf',1142182525315015008,94597,58896.182523,39.709609,36.789589,0.616080,0.707654,0.345936,15.6018,1.0,733,0.000004,39.709351,36.789603,0.616082,0.707651,0.345937
5207585,b'ZTF20aapdbbp',1145110625315015005,95182,58899.110625,40.456356,36.751514,0.609662,0.715006,0.342168,15.2027,1.0,733,0.000003,40.456109,36.751523,0.609665,0.715003,0.342169


In [33]:
ztf

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob
0,b'ZTF18acebhfp',676397301515010013,14490,58430.397303,41.357345,58.879488,0.387942,0.653853,0.649598,18.946699,0.865682
1,b'ZTF18abodmwk',596403415715010014,5831,58350.403414,30.969721,65.305308,0.358224,0.558644,0.748059,19.010401,0.855504
2,b'ZTF18abodmwk',626428345715010011,10614,58380.428345,30.969705,65.305294,0.358224,0.558644,0.748059,18.935900,0.855504
3,b'ZTF18abodmwk',630507595715015045,11250,58384.507593,30.969940,65.305305,0.358223,0.558645,0.748059,19.260401,0.855504
4,b'ZTF18abodmwk',618384965715010022,9040,58372.384965,30.969643,65.305179,0.358226,0.558644,0.748058,19.220200,0.855504
...,...,...,...,...,...,...,...,...,...,...,...
5459014,b'ZTF20aareruw',1151532523515015015,97109,58905.532523,253.007910,55.485537,-0.165587,-0.169403,0.971537,19.192400,0.608023
5459015,b'ZTF20aarerwx',1151533002615015009,97110,58905.533009,232.886408,53.509617,-0.358833,-0.115301,0.926253,19.687099,0.559474
5459016,b'ZTF20aarerww',1151533002115010003,97110,58905.533009,236.167899,54.618457,-0.322375,-0.116973,0.939357,19.957001,0.392662
5459017,b'ZTF20aarervr',1151526063515015015,97098,58905.526065,286.235286,33.876902,0.232120,-0.509626,0.828494,19.049299,0.517241


In [32]:
ztf.iloc[0]

ObjectID            b'ZTF18acebhfp'
CandidateID      676397301515010013
TimeStampID                   14490
mjd                         58430.4
ra                          41.3573
dec                         58.8795
ux                         0.387942
uy                         0.653853
uz                         0.649598
mag_app                     18.9467
asteroid_prob              0.865682
Name: 0, dtype: object

In [82]:
ztf_ast.loc[0]

ObjectID               b'ZTF18acebhfp'
CandidateID         676397301515010013
TimeStampID                      14490
mjd                            58430.4
ra                             41.3573
dec                            58.8795
ux                            0.387942
uy                            0.653853
uz                            0.649598
mag_app                        18.9467
asteroid_prob                 0.865682
nearest_ast_num                1208789
nearest_ast_dist             0.0050294
ast_ra                         41.3964
ast_dec                         58.592
ast_ux                        0.390926
ast_uy                        0.655662
ast_uz                        0.645976
Name: 0, dtype: object

In [78]:
fname = '../data/ztf_ast/ztf-nearest-ast.h5'

In [79]:
ztf_ast.to_hdf(fname, key='ztf_ast', mode='w')